<a href="https://colab.research.google.com/github/Muhammad-Usama-07/ComputerVision/blob/master/ComputerVisionWorkSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save all images of video 

In [ ]:
import cv2
camera = cv2.VideoCapture("NewObjectDetector.mp4")
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
counter = 0

while camera.isOpened():
    ret, frame = camera.read()
    if ret:
        
        try:
            image = cv2.resize(frame, (960, 540))
            print(f'Saving Image No. {counter}')
            cv2.imwrite(f'Data/{counter}.bmp', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except:
            pass
        counter +=1
    else:
        print('Frame not available')
        break


# Access camera with opencv and tkinter 

In [ ]:
# Import required Libraries
from tkinter import *
import tkinter.font as font
from PIL import Image, ImageTk
import cv2

# Create an instance of TKinter Window or frame
win = Tk()
def quit():
        win.destroy()

# Create a Label to capture the Video frames
label =Label(win)
label.grid(row=0, column=0)
cap= cv2.VideoCapture(0)

# Define function to show frame
def show_frames():
   # Get the latest frame and convert into Image
   cv2image= cv2.cvtColor(cap.read()[1],cv2.COLOR_BGR2RGB)
   img = Image.fromarray(cv2image)
   # Convert image to PhotoImage 
   imgtk = ImageTk.PhotoImage(image = img)
   label.imgtk = imgtk
   label.configure(image=imgtk)
   # Repeat after an interval to capture continiously
   label.after(20, show_frames)


show_frames()


myFont = font.Font(family='Helvetica', size=15)

button = Button(win, text = 'Click and Quit',  font=myFont ,command=quit)
button.place(relx = 0.4, rely = 0.9, anchor = CENTER)

win.mainloop()



